**如何对大量的短文本数据进行高效建模？**
**在LDA建模时，如何确定主题数？**
**主题模型得到的结果解释性程度不高、看不懂咋办？**

在没有任何类别或标签的情况下，利用无监督技术来提取文档主题是一个自然而然的想法，虽然LDA和NMF等主题模型已经被广泛采用，而且在大多时候效果还不错（主要是长文本），但是，笔者总觉得通过超参数调优来发掘有意义的话题需要花费相当大的精力，而且很多时候吃力不讨好---出来的结果奇差无比，上面的几个问题也印证了这一点。

很自然，我们想结合时下SOA的BERT---因为它在近两年的各种NLP任务中表现优异，而且使用预训练模型不需要有标注的数据，更重要的是BERT可以产生出高质量的、带有上下文语境信息的词嵌入和句嵌入。
接下来将以一个汽车行业的用户评论语料作为示例，展示基于bert模型的主题模型的强大威力。

## 其中使用的pyltp必须创建python3.6的环境，否则会报错

### 1 载入需要的python库

In [1]:
import numpy as np
import pandas as pd
import jieba
import umap
import hdbscan
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import matplotlib.pyplot as plt


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


### 2 载入数据，并作必要的数据预处理
笔者在这里使用的是汽车之家的口碑评论数据，有20000+，大部分是长度不超过70的短文本数据。

In [2]:
#我是用的是python3.7，因此pyltp使用不了
#据说这个库非常好用
# from pyltp import SentenceSplitter
# data = pd.read_excel('car_reviews.xlsx')
# data[['review']]= data[['review']].values.astype(str)
# splited_sentences =  SentenceSplitter.split(' '.join(data['review'].tolist()))
# data = pd.DataFrame(list(splited_sentences), columns=["review"])

In [3]:
# from pyltp import SentenceSplitter
data=pd.read_csv('data/train_2.csv')
data

,content_id,content,subject,sentiment_value,sentiment_word
0,13149,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,2288,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,1652,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,8865,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,11784,17价格忒高，估计也就是14-15左右。,价格,-1,高
...,...,...,...,...,...
12567,17392,全时四驱仅比一般SUV车强一点，肯定干不过Q5，XC60，连A4也干不过。水平对置仅体现在低...,动力,0,NaN
12568,9780,哈哈，终于看到有人开始厌烦前置雷达的声音了，这个亲，那个声音来自哪里？,配置,-1,NaN
12569,1079,请教一下，变速箱油，差速器油，火花塞，分别多久更换。,动力,0,NaN
12570,16766,求购二手１４款ＸＴ的后刹车总成。（已网购到手了）,安全性,0,NaN


In [4]:
import jieba
data['content']=data['content'].values.astype(str)
data=pd.DataFrame(list(data['content']),columns=["review"])

In [5]:
data['text_length']=data["review"].apply(lambda x:len(x))
print(len(data))
data=data[data['text_length']>5]
print(len(data))
data

12572
12572


,review,text_length
0,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,40
1,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,45
2,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,52
3,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,40
4,17价格忒高，估计也就是14-15左右。,23
...,...,...
12567,全时四驱仅比一般SUV车强一点，肯定干不过Q5，XC60，连A4也干不过。水平对置仅体现在低...,74
12568,哈哈，终于看到有人开始厌烦前置雷达的声音了，这个亲，那个声音来自哪里？,35
12569,请教一下，变速箱油，差速器油，火花塞，分别多久更换。,26
12570,求购二手１４款ＸＴ的后刹车总成。（已网购到手了）,24


对文本数据进行分词处理，同时排除语句中的停用词。

**注意，这个操作是作获取主题词用，生成语句表示，也就是句嵌入则是基于BERT模型。**

In [6]:
data.head()

,review,text_length
0,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,40
1,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,45
2,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,52
3,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,40
4,17价格忒高，估计也就是14-15左右。,23


In [7]:
from nltk.corpus import stopwords
my_stopwords=set(stopwords.words('english'))
my_stopwords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [8]:
data['review_seg']=data['review'].apply(lambda x:' '.join([j.strip() for j in jieba.lcut(x) if j not in my_stopwords]))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Eric\AppData\Local\Temp\jieba.cache
Loading model cost 1.732 seconds.
Prefix dict has been built successfully.


In [9]:
data.head()

,review,text_length,review_seg
0,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,40,因为 森林 人 即将 换代 ， 这套 系统 没 必要 装在 一款 即将 换代 的 车型 上 ...
1,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,45,四驱 价格 貌似 挺 高 的 ， 高 的 可以 看齐 XC60 了 ， 看实车 前 脸 有点...
2,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,52,斯柯达 要说 质量 ， 似乎 比 大众 要 好 一点 ， 价格 也 低 一些 ， 用料 完全...
3,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,40,这 玩意 都 是 给 有钱 任性 又 不 懂车 的 土豪 用 的 ， 这 价格 换 一次 我...
4,17价格忒高，估计也就是14-15左右。,23,17 价格 忒 高 ， 估计 也 就是 14 - 15 左右 。


### 3 创建高质量的语句嵌入
第一步是将文档向量化，且要尽量减少数据转换过程中带来的语义损失。有很多方法可以使用，比如doc2vec、skip-thought、elmo等，但考虑到 BERT 的优异特性，笔者此次用它来提取语句嵌入。

我们首先使用SentenceTransformer从一组文档中创建文档嵌入。如果针对目标领域进行了专有化的预训练，比如汽车领域的主题建模就用大量汽车领域的无标注语料finetune模型，那么语句嵌入的效果将会很好，能够将语义相近但表达不同的文本聚在一起，尤其是短文本这类难以处理的文档类型。

如果你有很长的文档，笔者则建议你利用工程手段把文档分成小段落或句子，因为SentenceTransformer是基于BERT模型，通常有语句长度限制，一般是512个字符。

In [10]:
model=SentenceTransformer(r'gao_dir/my_pretrained_chinese_embedding')

Exception when trying to download https://sbert.net/models/gao_dir/my_pretrained_chinese_embedding.zip. Response 404


OSError: Can't load config for 'gao_dir/my_pretrained_chinese_embedding'. Make sure that:

- 'gao_dir/my_pretrained_chinese_embedding' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'gao_dir/my_pretrained_chinese_embedding' is the correct path to a directory containing a config.json file

